# Pytorch + resnetrs + wandb

Basic pytorch resnetrs notebook

Current parameters
---------------------
1) Augmentation:
- Train:
    - resize
    - verticle flip
    - horizontal flip
    - tensorize
    
- Val/ Test
    - resize
    - tensorize

2) Model:
- resnetrs50(pretrained) (https://github.com/nachiket273/pytorch_resnet_rs)
- drop_rate = 0.1

3) Other parameters:
- image size = 320
- base lr = 1e-4
- minimum lr = 1e-7
- batch size = 32
- weight decay = 1e-3
- optimizer = AdamW
- loss = CrossEntropyLoss with class weights
- schedular = CosineAnnealingLr

4) Metrics:
- roc auc score
- F1 score
- accuracy

5) We use Exponential moving average of model parameters with decay rate of 0.995

6) we use weights and biases to track the runs.

In [ ]:
%matplotlib inline

Using my implementation of resnetrs.<br>
The repository is available @ https://github.com/nachiket273/pytorch_resnet_rs<br>

In [ ]:
!git clone https://github.com/nachiket273/pytorch_resnet_rs

In [ ]:
!cp -r pytorch_resnet_rs/* .

In [ ]:
!pip install wandb --upgrade

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import math
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import random
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.models as models

from tqdm.notebook import tqdm

from model import ResnetRS
from model.ema import EMA

In [ ]:
import wandb
wandb.login()

In [ ]:
print("Torch version: ", torch.__version__)
print("Torchvision version: ", torchvision.__version__)

# Load Data

In [ ]:
DIR = '../input/seti-breakthrough-listen/'
TRAIN_CSV = os.path.join(DIR, 'train_labels.csv')
TEST_CSV = os.path.join(DIR, 'sample_submission.csv')

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [ ]:
train_df.head()

In [ ]:
def get_img_path(img_id, train=True):
    if train:
        dir_path = os.path.join(DIR, 'train')
    else:
        dir_path = os.path.join(DIR, 'test')
    file_name = os.path.join(img_id[0], img_id + '.npy')
    dir_path = os.path.join(dir_path, file_name)
    return dir_path

In [ ]:
train_df['img_path'] = train_df['id'].apply(lambda k : get_img_path(k))
train_df.head()

In [ ]:
test_df['img_path'] = test_df['id'].apply(lambda k: get_img_path(k, False))
test_df.head()

# Config

In [ ]:
CFG = {
    'seed' : 37,
    'epochs' : 20,
    'img_size' : 320,
    'bs' : 32,
    'num_classes' : 2,
    'base_lr' : 1e-4,
    'weight_decay' : 1e-3,
    'test_size' : 0.1,
    'drop_rate'  : 0.1,
    'min_lr' : 1e-7
}

# Seed

In [ ]:
def seed_everything(seed=17):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything(CFG['seed'])

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

# Train data

In [ ]:
sns.countplot(x= train_df['target'])

In [ ]:
train_df['target'].value_counts(normalize=True)

### Show Cadence 

In [ ]:
def show_images(path, label):
    _id = path.split('/')[-1].split('.')[0]
    cadence = np.load(path).astype(np.float32)
    fig, ax = plt.subplots(nrows = 6, ncols = 1, figsize = (16, 10))
    fig.suptitle(f'ID:{_id}   TARGET:{label}', fontsize = 18)
    for i in range(6):
        ax[i].imshow(cadence[i], interpolation = 'nearest', aspect = 'auto')
        ax[i].text(5, 100, ["ON", "OFF"][i % 2], bbox={'facecolor': 'white'})
    
    plt.show()

### Training Data: Target = 0

In [ ]:
show_images(train_df[train_df['target'] == 0]['img_path'].values[0], 0)

### Training Data: Target = 1

In [ ]:
show_images(train_df[train_df['target'] == 1]['img_path'].values[0], 1)

### Test Data 

In [ ]:
test_sample = test_df.sample(3)
test_sample.head()

In [ ]:
for path in test_sample['img_path'].values:
    show_images(path, None)

# Custom Dataset

In [ ]:
class SetiDS(Dataset):
    def __init__(self, img_paths, labels, test=False, transform=None):
        self.test = test
        self.img_paths = img_paths 
        if not self.test:
            self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        file_path = self.img_paths[idx]
        
        image = np.load(file_path)
        image = image.astype(np.float32)
        image = np.vstack(image).transpose((1, 0))
        if self.transform is not None:
            image = self.transform(image=image)['image']
            #image = self.transform(image)
        else:
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()
            
        if not self.test:
            label = torch.tensor(self.labels[idx]).long()
            return image, label
        else:
            return image

In [ ]:
def get_transforms(train=True):
    if train:
        transform = A.Compose([
            A.Resize(CFG['img_size'], CFG['img_size']),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            ToTensorV2(),
        ])
    else:
        transform = A.Compose([
            A.Resize(CFG['img_size'], CFG['img_size']),
            ToTensorV2()
        ])
    return transform

In [ ]:
train_tf = get_transforms()
val_tf = get_transforms(False)
test_tf = get_transforms(False)

### Datasets and Dataloader

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_df['img_path'], train_df['target'],
                                                  test_size=CFG['test_size'],
                                                  random_state=CFG['seed'],
                                                  stratify=train_df['target'])

In [ ]:
train_ds = SetiDS(X_train.values, y_train.values, transform=train_tf)
val_ds = SetiDS(X_val.values, y_val.values, transform=val_tf)
test_ds = SetiDS(test_df['img_path'].values, None, test=True, transform=test_tf)

In [ ]:
train_dl = DataLoader(train_ds, batch_size=CFG['bs'], num_workers=4,
                      drop_last=False, shuffle=True, pin_memory=True)
valid_dl = DataLoader(val_ds, batch_size=CFG['bs'], num_workers=4,
                      drop_last=False, shuffle=False, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size=CFG['bs'], num_workers=4,
                     drop_last=False, shuffle=False, pin_memory=True)

# Helper Functions

In [ ]:
def save_checkpoint(model, filename='data/checkpoint.pth'):
    """Save checkpoint if a new best is achieved"""
    torch.save(model.state_dict(), filename)

In [ ]:
def load_checkpoint(model, filename = 'data/checkpoint.pth'):
    sd = torch.load(filename, map_location=lambda storage, loc: storage)
    names = set(model.state_dict().keys())
    for n in list(sd.keys()):
        if n not in names and n+'_raw' in names:
            if n+'_raw' not in sd: sd[n+'_raw'] = sd[n]
            del sd[n]
    model.load_state_dict(sd)

In [ ]:
class AvgStats(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.losses =[]
        self.acc = []
        self.prec = []
        self.rec = []
        self.F1 =[]
        self.roc_auc = []
        self.its = []
        
    def append(self, loss, acc, prec, rec, F1, roc_auc, it):
        self.losses.append(loss)
        self.acc.append(acc)
        self.prec.append(prec)
        self.rec.append(rec)
        self.F1.append(F1)
        self.roc_auc.append(roc_auc)
        self.its.append(it)

# Train and Test

In [ ]:
def train(loader, model, optimizer, criterion, ema, device):
    model.train()
    running_loss = 0.
    correct = 0
    start_time = time.time()
    t = tqdm(loader, leave=False, total=len(loader))
    preds, tgts = list(), list()

    for i, (ip, tgt) in enumerate(t):
        ip, tgt = ip.to(device), tgt.to(device)
        output = model(ip)
        loss = criterion(output, tgt)
        running_loss += loss.item()
        _, pred = output.max(dim=1)
        correct += torch.sum(pred == tgt.data)
        tgt = tgt.cpu().detach().numpy()
        pred = pred.cpu().detach().numpy()
        tgts.extend(list(tgt))
        preds.extend(list(pred))
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        ema.update(model.parameters())
        
    trn_time = time.time() - start_time        
    trn_losses = running_loss /len(loader)
    trn_acc = correct * 100/ len(loader.dataset)
    trn_F1 = f1_score(tgts, preds, average='weighted', labels=np.unique(tgts)) * 100
    trn_prec = precision_score(tgts, preds, average='weighted', labels=np.unique(tgts)) * 100
    trn_rec = recall_score(tgts, preds, average='weighted', labels=np.unique(tgts)) * 100
    trn_roc_auc = roc_auc_score(tgts, preds, average='weighted', labels=np.unique(tgts)) * 100
    
    return trn_time, trn_F1, trn_losses, trn_acc, trn_prec, trn_rec, trn_roc_auc

In [ ]:
def test(loader, model, criterion, ema, device):
    with torch.no_grad():
        model.eval()
        running_loss = 0.
        correct = 0
        start_time = time.time()
        t = tqdm(loader, leave=False, total=len(loader))
        preds, tgts = list(), list()
        
        for i, (ip, tgt) in enumerate(t):
            ip, tgt = ip.to(device), tgt.to(device)
            ema.store(model.parameters())
            ema.copy(model.parameters())
            output = model(ip)
            loss = criterion(output, tgt)
            ema.copy_back(model.parameters())
            running_loss += loss.item()
            _, pred = output.max(dim=1)
            correct += torch.sum(pred == tgt.data)
            tgt = tgt.cpu().detach().numpy()
            pred = pred.cpu().detach().numpy()
            tgts.extend(list(tgt))
            preds.extend(list(pred))
            
        val_time = time.time() - start_time
        val_losses = running_loss/len(loader)
        val_acc = correct * 100 / len(loader.dataset)
        val_F1 = f1_score(tgts, preds, average='weighted', labels=np.unique(tgts)) * 100
        val_prec = precision_score(tgts, preds, average='weighted', labels=np.unique(tgts)) * 100
        val_rec = recall_score(tgts, preds, average='weighted', labels=np.unique(tgts)) * 100
        val_roc_auc = roc_auc_score(tgts, preds, average='weighted', labels=np.unique(tgts)) * 100
        
        return val_time, val_F1, val_losses, val_acc, val_prec, val_rec, val_roc_auc

In [ ]:
def fit(model, epochs, optimizer, criterion, ema, device, wandb, sched=None):
    best_roc_auc = 0.
    print("Epoch\tTrn_loss\tVal_loss\tTrn_roc_auc\tVal_roc_auc\tTrn_Acc\t\tVal_Acc")
    limit = 0
    for j in range(epochs):
        trn_time, trn_F1, trn_losses, trn_acc, trn_prec, trn_rec, trn_roc_auc = train(train_dl, model, optimizer, criterion, ema, device)
        train_stats.append(trn_losses, trn_acc, trn_prec, trn_rec, trn_F1, trn_roc_auc, trn_time)
        val_time, val_F1, val_losses, val_acc, val_prec, val_rec, val_roc_auc = test(valid_dl, model, criterion, ema, device)
        test_stats.append(val_losses, val_acc, val_prec, val_rec, val_F1, val_roc_auc, val_time)
        wandb.log({
            "Train Loss": trn_losses,
            "Valid Loss": val_losses,
            "Train Acc": trn_acc,
            "Valid Acc": val_acc,
            "Train Roc Auc Score": trn_roc_auc,
            "Valid Roc Auc Score": val_roc_auc,
            "Train Precision Score": trn_prec,
            "Valid Precision Score": val_prec,
            "Train Recall Score": trn_rec,
            "Valid Recall Score": val_rec,
            "Train F1 Score": trn_F1,
            "Valid F1 Score": val_F1
        })
        if(val_roc_auc > best_roc_auc):
            limit = 0
            best_roc_auc = val_roc_auc
            save_checkpoint(model, './best_model.pth')
        else:
            limit += 1
            if limit == 5:
                break
        if sched:
            sched.step()
        print("{}\t{:06.8f}\t{:06.8f}\t{:06.8f}\t{:06.8f}\t{:06.8f}\t{:06.8f}"
              .format(j+1, trn_losses, val_losses, trn_roc_auc, val_roc_auc, trn_acc, val_acc))
    wandb.run.summary["Best Roc Auc Score"] = best_roc_auc

# Model and Training

In [ ]:
def get_model():
    model = ResnetRS.create_pretrained('resnetrs50', in_ch=1, num_classes=2,
                                       drop_rate=CFG['drop_rate'])
    for param in model.parameters():
        param.require_grad = True
    model = model.to(device)
    return model

In [ ]:
model = get_model() 

In [ ]:
save_checkpoint(model, './init.pth')

## Criterion, optimizer and scheduler

In [ ]:
weights = torch.tensor([0.9, 0.1]).to(device)

criterion = nn.CrossEntropyLoss(weights).to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['base_lr'],
                              weight_decay=CFG['weight_decay'])

In [ ]:
sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, CFG['epochs'],
                                                   eta_min=CFG['min_lr'])

In [ ]:
ema = EMA(model.parameters(), decay_rate=0.995, num_updates=0)

## Fit and wandb tracking

In [ ]:
train_stats = AvgStats()
test_stats = AvgStats()

In [ ]:
run = wandb.init(project='Seti-ResNetRS', config=CFG,
                 group = 'resnetrs', job_type='train',
                 name = 'resnetrs50_ema_wandb')

In [ ]:
wandb.watch(model)

In [ ]:
print(f"{'='*25} Fit {'='*25}")
fit(model, CFG['epochs'], optimizer, criterion, ema, device, wandb, sched=sched)

In [ ]:
def plot(train_stats, test_stats):
    nrows, ncols = 3, 2
    fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = (16, 10))

    # Accuracy
    ax[0, 0].plot(train_stats.acc, label='train')
    ax[0, 0].plot(test_stats.acc, label='val')
    ax[0, 0].set_title("Accuracy")
    ax[0, 0].legend(loc='upper left')

    # Roc_Auc_Score
    ax[0, 1].plot(train_stats.roc_auc, label='train')
    ax[0, 1].plot(test_stats.roc_auc, label='val')
    ax[0, 1].set_title("Roc Auc Score")
    ax[0, 1].legend(loc='upper left')

    # Precision score
    ax[1, 0].plot(train_stats.prec, label='train')
    ax[1, 0].plot(test_stats.prec, label='val')
    ax[1, 0].set_title("Precision Score")
    ax[1, 0].legend(loc='upper left')

    # Recall score
    ax[1, 1].plot(train_stats.rec, label='train')
    ax[1, 1].plot(test_stats.rec, label='val')
    ax[1, 1].set_title("Recall Score")
    ax[1, 1].legend(loc='upper left')

    # F1 score
    ax[2, 0].plot(train_stats.F1, label='train')
    ax[2, 0].plot(test_stats.F1, label='val')
    ax[2, 0].set_title("F1 Score")
    ax[2, 0].legend(loc='upper left')

    # Loss
    ax[2, 1].plot(train_stats.losses, label='train')
    ax[2, 1].plot(test_stats.losses, label='val')
    ax[2, 1].set_title("Losses")
    ax[2, 1].legend(loc='upper left')


    plt.show()

In [ ]:
plot(train_stats, test_stats)

# Load Best saved parameters

In [ ]:
load_checkpoint(model, './best_model.pth')

# Predict

In [ ]:
def get_predicts(loader, model, device):
    preds = list()
    t = tqdm(loader, leave=False, total=len(loader))
    with torch.no_grad():
        model.eval()
        for i, ip in enumerate(t):
            ip = ip.to(device)
            output = model(ip)
            _, pred = output.max(dim=1)
            pred = pred.cpu().detach().numpy()
            preds.extend(list(pred))
    return preds

In [ ]:
preds = get_predicts(test_dl, model, device)

In [ ]:
test_df['target'] = preds
test_df.drop(['img_path'], axis=1, inplace=True)
test_df.to_csv('submission.csv', index=False)

In [ ]:
run.finish()